<a href="https://colab.research.google.com/github/krajesh00/Data_Platform_Co-op_Case_Study/blob/master/Data_Platfrom_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
!pip install pyspark
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark
# Import a Spark function from library
from pyspark.sql.functions import *




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [108]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import udf 
from pyspark.sql.window import Window
import pandas as pd
# Read the string data into a DataFrame
data = 'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'
df = spark.read.csv(spark.sparkContext.parallelize(data.split("\n")), header=True, sep=';')

# 1. Inputting Flight Codes for missing values and changing data types of all elements to integer
flight_codes = df.rdd.map(lambda x: x[2]).collect()

for i in range (0, len(flight_codes)):
  if flight_codes[i] is None:
     flight_codes[i] = int(float(flight_codes[i-1]) + 10)
  else:
    flight_codes[i] = int(float(flight_codes[i]))


w = Window().orderBy(lit('A'))
df = df.withColumn('serial_number', row_number().over(w))

replacement = udf(lambda x : flight_codes[x-1])
df = df.withColumn("FlightCodes", replacement(df.serial_number)).drop('serial_number')

# 2. Split the data from the To_From column into two different and make them capital case
df = df.withColumn("From", split(df["To_From"], "_").getItem(0))
df = df.withColumn("To", split(df["To_From"], "_").getItem(1))
df = df.drop("To_From")
df = df.withColumn("From", upper(col("From")))
df = df.withColumn("To", upper(col("To")))

#3. Cleaning up airline codes
df = df.withColumn("Airline Code", regexp_replace(col("Airline Code"), "[^a-zA-Z ]", ""))

# Final table
df.show()


+---------------+------------+-----------+--------+---------+
|   Airline Code|  DelayTimes|FlightCodes|    From|       To|
+---------------+------------+-----------+--------+---------+
|    Air Canada |    [21, 40]|      20015|WATERLOO|  NEWYORK|
|    Air France |          []|      20025|MONTREAL|  TORONTO|
|Porter Airways |[60, 22, 87]|      20035| CALGARY|   OTTAWA|
|     Air France|    [78, 66]|      20045|  OTTAWA|VANCOUVER|
|      Lufthansa|    [12, 33]|      20055|  LONDON| MONTREAL|
+---------------+------------+-----------+--------+---------+

